# Create 3D Boundary Conditions of the 79NG Fjord GETM Setup

This notebook creates the 3D boundary conditions of the 79NG fjord setup in GETM.

The data used as boundary conditions have been provided by Claudia Wekerle (AWI).
They come from the FESOM2 setup with increased resolution in the 79NG fjord used by McPherson, Wekerle and Kanzow (2023, https://doi.org/10.1029/2023JC019915).

Notebook by Markus Reinert (IOW, 2023, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from pyproj import CRS, Transformer

## Prepare the coordinate transformation

The interpolation of the data for the boundary conditions must be computed on a Cartesian coordinate system, not a latitude–longitude system, in order to have the correct distances between grid points.
Here we define
* the Coordinate Reference System (CRS) in which the model grids are given,
* a projected CRS that is suitable for the interpolation,
* a Transformer object from the former to the latter.

### CRS of model grids

In [ ]:
crs_latlon = CRS.from_epsg(4326)
crs_latlon

### CRS for interpolation

In [ ]:
crs_cartesian = CRS.from_epsg(3413)
crs_cartesian

### CRS transformer

In [ ]:
transformer = Transformer.from_crs(crs_latlon, crs_cartesian)

## Load the FESOM data

In [ ]:
# Idea for generalization: loop over all years,
# for each year load all NetCDF-files for this year,
# and combine them into one xarray Dataset,
# applying the mask obtained from zero salinity
year = 2010
var = "salt"
filename = f"data/FESOM/{var}.fesom.{year}.sub.nc"
fesom = xr.open_dataset(filename)

# Mask out cells with zero salinity
fesom["mask"] = fesom.salt > 0
fesom[var] = fesom[var].where(fesom.mask)

# Remove layers where all values are masked out
for k in range(fesom.nz1.size):
    if not np.any(fesom.mask[:, :, k]):
        assert np.all(fesom.mask.isel(nz1=slice(k, None)) == False), "not all deeper levels are masked"
        assert fesom.nz1.size == fesom.nz.size - 1, "dimension nz is not 1 larger than nz1"
        fesom = fesom.isel(nz1=slice(k), nz=slice(k+1))
        break

# Add Cartesian coordinates to the dataset
fesom.coords["X"], fesom.coords["Y"] = (
    ("nod2", coord, {"long_name": axis_info.name, "units": "m", "CRS": str(crs_cartesian)})
    for coord, axis_info in zip(transformer.transform(fesom.lat, fesom.lon), crs_cartesian.axis_info)
)

del fesom["faces"]

fesom

## Define the target grid

In [ ]:
getm = xr.Dataset({"lon": np.arange(-20, -14.9, 0.025)})
getm["lat"] = 79.2 * np.ones_like(getm.lon)
getm["X"], getm["Y"] = transformer.transform(getm.lat, getm.lon)
getm

## Show the grids

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12, 5), dpi=200, constrained_layout=True)

for ax, x, y, crs in zip(axs, ["lon", "X"], ["lat", "Y"], [crs_latlon, crs_cartesian]):
    ax.scatter(fesom[x], fesom[y], 1, fesom.mask.isel(time=0, nz1=0))
    ax.scatter(getm[x], getm[y], 1, "r")
    ax.set_title(f"{crs.name}\n{crs}")
    ax.set_xlabel(f"{fesom[x].long_name} [{fesom[x].units}]")
    ax.set_ylabel(f"{fesom[y].long_name} [{fesom[y].units}]")
ax.set_aspect("equal")

## Interpolate the data

In [ ]:
assert fesom.time.size == 12, "FESOM dataset does not have 12 months"
salt = xr.DataArray(np.full((fesom.nz1.size, getm.lon.size), np.nan), [-fesom.nz1, getm.lon])
for k in range(fesom.nz1.size):
    for month in range(12):
        salt[k] = griddata((fesom.X, fesom.Y), fesom.salt[month, :, k], (getm.X, getm.Y))
        break  # TODO: remove (this is only here to show a result quickly)
salt.plot()